In [35]:
!pip install requests_oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 KB 3.3 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 37.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 KB 17.6 MB/s eta 0:00:00


In [6]:
!pip install webdriver_manager

In [1]:
import re
import requests_cache
import deviantart
from bs4 import BeautifulSoup
import requests
import re, time, random
import pandas as pd
import io
import os, json
import deviantart

In [15]:
da = deviantart.Api("42096", "97080792c6d30a4178965e41f1ca15de")

def get_watchers_and_watching(username):
        """Gathers watchers and watching using API."""
        try:
            # Get watchers
            # Initialize an empty list to store watchers
            watchers_list = []
            
            # Get the initial batch of watchers
            watchers = da.get_watchers(username)
            
            # Add the usernames to the list
            for watcher in watchers['results']:
                watchers_list.append(watcher['user'].username)
            
            # Check if there are more watchers to fetch
            has_more = watchers['has_more']
            
            while has_more:
                # Get the next batch of watchers using the next_offset
                watchers = da.get_watchers(username, offset=watchers['next_offset'])
            
                # Add the usernames to the list
                for watcher in watchers['results']:
                    watchers_list.append(watcher['user'].username)
            
                # Update has_more for the next iteration
                has_more = watchers['has_more']
            
    
            # Get watching (friends)
            watching_list = []
    
            # Get the initial batch of watching users
            watching = da.get_friends(username)  # Use get_friends() to get watching list
            
            # Add the usernames to the list
            for watched in watching['results']:
                watching_list.append(watched['user'].username)
            
            # Check if there are more users to fetch
            has_more = watching['has_more']
            
            while has_more:
                # Get the next batch of users using the next_offset
                watching = da.get_friends(username, offset=watching['next_offset'])
            
                # Add the usernames to the list
                for watched in watching['results']:
                    watching_list.append(watched['user'].username)
            
                # Update has_more for the next iteration
                has_more = watching['has_more']
    
        
        except requests.exceptions.RequestException as e:
            print(f"Error scraping DeviantArt watchers and watching: {e}")
            print(f"Entering HTTPError handler...")
            print(f"HTTP Error: {e} for user: {username}")
            status_code_match = re.search(r"(\d{3})", str(e))  # Extract status code from error message
            if status_code_match and status_code_match.group(1) == "401":
                print("401 Unauthorized error. Reauthenticating...")
                reauth = da.auth()
            elif e.response.status_code == 403:
                print(f"HTTP 403 Error for user {username}. Access forbidden.")
                reauth = da.auth()
            #if e.response.status_code == 401:
             #   print("401 Unauthorized error detected. Reauthenticating...")
             #   reauthenticate()  # Call the reauthentication function
            else:
                # Handle other HTTP errors
                print(f"Handling other HTTP errors...")
                

        return watchers_list, watching_list

In [16]:
da = deviantart.Api("42096", "97080792c6d30a4178965e41f1ca15de")
requests_cache.install_cache('deviantart_cache', backend='sqlite', expire_after=3600)
def get_watchers_and_watching(username):
        """Gathers watchers and watching using API."""
        try:
            # Get watchers
            # Initialize an empty list to store watchers
            watchers_list = []
            
            # Get the initial batch of watchers
            watchers = da.get_watchers(username)
            
            # Add the usernames to the list
            for watcher in watchers['results']:
                watchers_list.append(watcher['user'].username)
            
            # Check if there are more watchers to fetch
            has_more = watchers['has_more']
            
            while has_more:
                # Get the next batch of watchers using the next_offset
                watchers = da.get_watchers(username, offset=watchers['next_offset'])
            
                # Add the usernames to the list
                for watcher in watchers['results']:
                    watchers_list.append(watcher['user'].username)
            
                # Update has_more for the next iteration
                has_more = watchers['has_more']
            
    
            # Get watching (friends)
            watching_list = []
    
            # Get the initial batch of watching users
            watching = da.get_friends(username)  # Use get_friends() to get watching list
            
            # Add the usernames to the list
            for watched in watching['results']:
                watching_list.append(watched['user'].username)
            
            # Check if there are more users to fetch
            has_more = watching['has_more']
            
            while has_more:
                # Get the next batch of users using the next_offset
                watching = da.get_friends(username, offset=watching['next_offset'])
            
                # Add the usernames to the list
                for watched in watching['results']:
                    watching_list.append(watched['user'].username)
            
                # Update has_more for the next iteration
                has_more = watching['has_more']
    
        
        except requests.exceptions.RequestException as e:
            print(f"Error scraping DeviantArt watchers and watching: {e}")
            print(f"Entering HTTPError handler...")
            print(f"HTTP Error: {e} for user: {username}")
            status_code_match = re.search(r"(\d{3})", str(e))  # Extract status code from error message
            if status_code_match and status_code_match.group(1) == "401":
                print("401 Unauthorized error. Reauthenticating...")
                reauth = da.auth()
            elif e.response.status_code == 403:
                print(f"HTTP 403 Error for user {username}. Access forbidden.")
            #if e.response.status_code == 401:
             #   print("401 Unauthorized error detected. Reauthenticating...")
             #   reauthenticate()  # Call the reauthentication function
            else:
                # Handle other HTTP errors
                print(f"Handling other HTTP errors...")
                

        return watchers_list, watching_list
# Load DataFrames from CSV files
#df1 = pd.read_csv(csv1_path)
#df2 = pd.read_csv(csv2_path)

#unique_usernames = df1[~df1['username'].isin(df2['username'])]['username'].tolist()
def deviants_watchers_watching(start_index, end_index, max_retries=3):
    df_usernames = pd.read_csv("/home/maitreyee/PycharmProjects/DataGatheringDeviantArt/Usernames_merged/merged_data2.csv")  # Replace with your CSV file
    output_file = "/home/maitreyee/PycharmProjects/DataGatheringDeviantArt/deviantart_deviants3LkWatch&ing.csv"  # Replace with your desired output file path
    batch_size = 10
    watcher_watching_data = []
    count = 0
    
    for i in range(start_index, min(end_index, len(df_usernames)), batch_size):
        batch_usernames = df_usernames['username'][i:i + batch_size]  # Get current batch
        for username in batch_usernames:
           # retries = 0
           # while retries < max_retries:
            try:
                    #statistics, groupmem, badges, about_info = scraper.scrape(username)
                    watchers, watching = get_watchers_and_watching(username)
                    watch_friends = {
                        "username": username,
                        "watchers": watchers,
                        "watching": watching
                    }
                    print(f"Successfully processed user: {username}")  # Print here
        
                    watcher_watching_data.append(watch_friends)
                    count += 1
        
                    # Dump to JSON after every 10 usernames
                    if count % 10 == 0:
                        df = pd.DataFrame(watcher_watching_data)
                        df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False, encoding='utf-8')
                        all_data = []  # Clear data list for the next batch
                        print(f"Dumped data for {count} usernames to {output_file}")
        
                    # Introduce a delay to avoid rate limiting
                    time.sleep(1)  # Wait for 1 second
        
            
                    if i + batch_size < len(df_usernames):  # Sleep if not the last batch
                        print("Sleeping for a minute...")
                        time.sleep(10)  # 5 minutes in seconds
    
        
            except Exception as e:
                print(f"Error processing user {username}: {e}")
                print(f"Entering HTTPError handler...")
                print(f"HTTP Error: {e} for user: {username}")
                 #   status_code_match = re.search(r"(\d{3})", str(e))  # Extract status code from error message
                 #   if status_code_match and status_code_match.group(1) == "401":
                 #       print("401 Unauthorized error. Reauthenticating...")
                 #       da = deviantart.Api("42096", "97080792c6d30a4178965e41f1ca15de")
                 #       retries += 1
                 #       continue  # Try again
                    #elif e.response.status_code == 403:
                     #   print(f"HTTP 403 Error for user {username}. Access forbidden.")
    
                    #else:
                     #   print(f"Error getting data for {username}: {e}")
                     #   return None  # Give up for other errors
        
            # Dump remaining data if any
        if watcher_watching_data:
            df = pd.DataFrame(watcher_watching_data)
            df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False, encoding='utf-8')
            print(f"Dumped remaining data to {output_file}")
        
                
        print("DeviantArt Api calling completed.")



deviants_watchers_watching(start_index=396, end_index=440)

FileNotFoundError: [Errno 2] No such file or directory: '/home/maitreyee/PycharmProjects/DataGatheringDeviantArt/Usernames_merged/merged_data2.csv'

In [13]:
username = "DigitalAIHeaven"

watchers, watching = get_watchers_and_watching(username)

print("Watchers:", watchers)
print("Watching:", watching)

Watchers: ['hau30ngai', 'hometownboy93', 'MythicArtery', 'ldeath101', 'vazmic', 'bow252740', 'yankeehawk', 'Dragoncobraking', 'Lauradog200', 'bth23041', 'JewelsArtCreations', 'CheeseBurger122', 'GodkingTyrant', 'OtakukingA7X', 'Mostaphi', 'KhatasCreationCorner', 'MathiasHeart', 'hypnoisthebest1234', 'isabeille32', 'PedroNeves2024onDA', 'LavenderBlueMoon', 'ai-nina', 'Iamthedayseye', 'OmniDog', 'CMC1232', 'ZeusZuu', 'nanahwang', 'HokuRyu', 'worldmusiclvr', 'Valqira', 'Shinelightart86', 'fantasiapearls', 'nautEnuf', 'PixAIdust', 'DjsHQskyCores', 'sableblu', 'rnjeetraja', 'Alex1372p', 'walebandz', 'Run3b0und', 'goldcharis', 'Phishn', 'renardparesseux', 'Deviantlore2024', 'ramsesdvart', 'zombiekila88', 'KiraHeat', 'salierimorella', 'thedatar', 'GodzillaLoverAwesome', 'Smokingmirror901', 'possumfish40', 'jakecollins81', 'J-T-Baker', 'apfelwolle', 'Provert10000', 'fatzfatz', 'o-chunks-kun', 'carlafritze', 'K2-REAPER', 'MoonLightUmbra1998', 'voidstudios', 'prince-of-today', 'elara1998', 'Fote

In [11]:
import deviantart
import requests
import random
import sqlite3
from bs4 import BeautifulSoup
import json
import time
import requests
import requests.auth
import datetime
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import threading

In [12]:
class DeviantArtRandomWalk:
    
    def __init__(self, client_id, client_secret, database_path="/mnt/hdd/maittewa/random_deviants_watchers_watching_testing.db"):
        self.da = deviantart.Api(client_id, client_secret)
        self.database_path = database_path
        self.create_database()
        self.conn = sqlite3.connect(self.database_path)
        self.cursor = self.conn.cursor()


    def get_token(self):
         client = BackendApplicationClient(client_id=client_id)
         oauth = OAuth2Session(client=client)
         token = oauth.fetch_token(token_url=TOKEN_URL, client_id=client_id, client_secret=client_secret)
         # Extract the access token
         access_token = token['access_token']
         return access_token
        
    def refresh_token(refresh_token, client_id, client_secret):
        """Refreshes the DeviantArt access token."""
        token_url = "https://www.deviantart.com/oauth2/token"  # DeviantArt token endpoint
        data = {
            "grant_type": "refresh_token",
            "refresh_token": refresh_token,
            "client_id": client_id,
            "client_secret": client_secret
        }

        try:
            response = requests.post(token_url, data=data)
            response.raise_for_status()  # Raise HTTPError for bad responses
            token_data = response.json()

            # Update token_data with new values
            token_data["access_token"] = token_data["access_token"]
            token_data["expires_at"] = int(time.time()) + token_data["expires_in"]  

            return token_data["access_token"]

        except requests.exceptions.RequestException as e:
            print(f"Error refreshing token: {e}")
            return None  # Or handle the error as needed

    def token_refresh_loop():
        """Periodically refreshes the token."""
        while True:
            time.sleep(20 * 60)  # Sleep for 20 minutes
            new_access_token = self.refresh_token(token_data["refresh_token"], client_id, client_secret)
            if new_access_token:
                token_data["access_token"] = new_access_token
                print("Token refreshed successfully.")
            else:
                print("Token refresh failed.")


    def random_days_since_2001():
        """Returns a random number of days since January 1, 2001."""
        today = datetime.date.today()
        start_date = datetime.date(2001, 1, 1)
        total_days = (today - start_date).days
        random_days = random.randint(0, total_days)
        return random_days
    
    def create_database(self):
        """Creates the SQLite database if it doesn't exist."""
        conn = sqlite3.connect(self.database_path)
        cursor = conn.cursor()

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS deviants (
                username TEXT PRIMARY KEY,
                profile_url TEXT
            )
        """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS relationships (
                deviant TEXT,
                related_deviant TEXT,
                relationship_type TEXT,  -- 'watcher' or 'watching'
                FOREIGN KEY (deviant) REFERENCES deviants(username),
                FOREIGN KEY (related_deviant) REFERENCES deviants(username)
            )
        """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS group_memberOf (
                username TEXT,
                name TEXT
            )
        """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS group_adminOf (
                username TEXT,
                name TEXT
            )
        """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS stats (
                key TEXT PRIMARY KEY,  
                value TEXT  
            )
         """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS profile (
                key TEXT PRIMARY KEY,  
                value TEXT  
            )
         """)

        conn.commit()
        conn.close()

    def get_random_deviants_from_daily_deviations(self, num_deviants, date):
        """Fetches a list of random deviants from a specific tag and page."""
        token = self.get_token() 
        url = f"https://www.deviantart.com/api/v1/oauth2/browse/dailydeviations?access_token={token}"  
        params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "date": date.strftime("%Y-%m-%d"),  # Format date as YYYY-MM-DD
        # ... (other parameters if needed, e.g., limit, offset) ...
        }
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
            data = response.json() 
            #results = self.da.browse_dailydeviations()  
            #results = self.da.browse(tag=tag, offset=(page - 1) * 24)  # 24 results per page by default
            deviants = [deviation["author"]["username"] for deviation in data["results"]]
            random_deviants = random.sample(deviants, min(num_deviants, len(deviants))) 
            return random_deviants
            # Get as many as possible
        except Exception as e:
            print(f"Error fetching deviants: {e}")
            return []

    def get_user_profile(self,username):
        token= self.get_token()
        #url=f"https://www.deviantart.com/api/v1/oauth2/user/profile/{username}?access_token={token}"
        url=f"https://www.deviantart.com/api/v1/oauth2/user/profile/{username}?access_token={token}"
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error getting profile info: {e}")
            return None
        return response

    def parse_user_profile(self, user):
        s= self.get_user_profile(user)
        d=json.loads(s.text)
        if 'error' not in d.keys():
            a={'username':user,
            'real_name': d.get('real_name'),
            'profil_url': d['profile_url'],
            'Artist or Not': d['user_is_artist'],   
            'tags': d['tagline'],
            'country': d['country'],
            'level': d['artist_level'],
            'specialty' :d['artist_specialty'],
            'stats': d['stats']}
            return a
            print(f'Gather profile information for the {user}')
        else:
            return 'error'
            
    def scrape(self, username):   
        try:
            # --- About Page ---
            about_page_url = f"https://www.deviantart.com/{username}/about"
            about_page_response = requests.get(about_page_url)
            about_page_response.raise_for_status()  # Raise an exception for bad responses
      
            about_page_soup = BeautifulSoup(about_page_response.content, "html.parser")
            id_pattern = re.compile(r"module-\d{10}")
            # Extract About page (adjust the selector based on the page structure)
            aboutpage = about_page_soup.select_one("div.wuU4s._3tnu8._2yPuh")
            userstats_element = about_page_soup.find(id="userstats")  # Userstats
            memberOf = about_page_soup.find(id="group_list_members")  
            adminOf = about_page_soup.find(id="group_list_admins")
            badges_element = about_page_soup.find(id="badges_activity")
            about_deviant = about_page_soup.find(id=id_pattern)

            userstats = userstats_element.text.strip() if userstats_element else ""
            badges = badges_element.text.strip() if badges_element else ""
            about_devts = about_deviant.text.strip() if about_deviant else ""
            groupmem = memberOf.text.strip() if memberOf else ""
            groupadmin = adminOf.text.strip() if adminOf else ""
            # Find the section with the specific ID
    
            member_hrefs_and_names = []
            admin_hrefs_and_names = []

                
            statistics_data = {}
    
            # Define patterns for each statistic
            patterns = {
            "Pageviews": r"Pageviews(\d+)",
            "Deviations": r"Deviations(\d+)",
            "Watchers": r"Watchers(\d+)",
            "Watching": r"Watching(\d+)",
            "Favourites": r"Favourites(\d+)",
            "Comments Made": r"Comments\sMade(\d+)",  # Handle space in "Comments Made"
            "Comments Received": r"Comments\sReceived(\d+)",  # Handle space in "Comments Received"
            }

            # Extract values using patterns
            for key, pattern in patterns.items():
              match = re.search(pattern, userstats)
              if match:
                statistics_data[key] = int(match.group(1))  # Convert value to integer

            print(statistics_data, groupmem, groupadmin, badges, about_devts)    
            return statistics_data, groupmem, groupadmin, badges, about_devts
    
        except requests.exceptions.RequestException as e:
            print(f"Error scraping DeviantArt profile: {e}")
            if "403 Client Error" in str(e):  # Check for Forbidden error
                print(f"Waiting for 60 seconds before retrying...")
                time.sleep(60)  # Wait for 60 seconds
                return self.scrape(username)  # Retry the request
            
    def get_watchers_and_watching(self, username, sample_size=10):
        """Gets the watchers and watching of a deviant."""
        watchers_list = []
        watching_list = []
        try:
          # Get watchers
            # Initialize an empty list to store watchers
            watchers_list = []
            
            # Get the initial batch of watchers
            watchers = self.da.get_watchers(username)
            
            # Add the usernames to the list
            for watcher in watchers['results']:
                watchers_list.append(watcher['user'])
            
            # Check if there are more watchers to fetch
            has_more = watchers['has_more']
            
            while has_more:
                # Get the next batch of watchers using the next_offset
                watchers = self.da.get_watchers(username, offset=watchers['next_offset'])
            
                # Add the usernames to the list
                for watcher in watchers['results']:
                    watchers_list.append(watcher['user'])
            
                # Update has_more for the next iteration
                has_more = watchers['has_more']
            
            # Get watching (friends)
            watching_list = []
    
            # Get the initial batch of watching users
            watching = self.da.get_friends(username)  # Use get_friends() to get watching list
            
            # Add the usernames to the list
            for watched in watching['results']:
                watching_list.append(watched['user'])
            
            # Check if there are more users to fetch
            has_more = watching['has_more']
            
            while has_more:
                # Get the next batch of users using the next_offset
                watching = self.da.get_friends(username, offset=watching['next_offset'])
            
                # Add the usernames to the list
                for watched in watching['results']:
                    watching_list.append(watched['user'])
            
                # Update has_more for the next iteration
                has_more = watching['has_more']

        except Exception as e:
            if "HTTP Error 429" in str(e):  # Check for HTTP 429 (Too Many Requests)
                print(f"Rate limit hit! Waiting for 60 seconds before retrying...")
                time.sleep(60)  # Wait for 60 seconds
                return self.get_watchers_and_watching(username, sample_size)  # Retry the request
            elif "HTTP Error 401" in str(e):  # Check for HTTP 429 (Too Many Requests)
                print(f"API returning unauthorised error. Refreshing the token")
                self.refresh_token()
                time.sleep(300)  # Wait for 300 seconds
                return self.get_watchers_and_watching(username, sample_size)  # Retry the request
            else:
                print(f"Error fetching watchers/watching for {username}: {e}")
                return [], []
        except Exception as e:
            print(f"Error fetching watchers/watching for {username}: {e}")
            return [], []

        return watchers_list, watching_list

        
    def store_data(self, profile): 
        """Stores the data in the SQLite database."""
        conn = sqlite3.connect(self.database_path)
        cursor = conn.cursor()

        cursor.execute("INSERT OR IGNORE INTO deviants (username, profile_url) VALUES (?, ?)",
                       (deviant, f"https://www.deviantart.com/{deviant}"))
        print("Storing data for deviant:", deviant)  # Add this print statement
        
        #for related_deviant in watchers:
         #   cursor.execute("INSERT INTO relationships VALUES (?, ?, ?)",
          #                 (deviant, related_deviant.username, "watcher"))

        #for related_deviant in watching:
         #   cursor.execute("INSERT INTO relationships VALUES (?, ?, ?)",
          #                 (deviant, related_deviant.username, "watching"))

        #for name in mem:
         #   cursor.execute("INSERT INTO group_memberOf (username, name) VALUES (?, ?, ?)", 
          #                 (username, name))

        #for href, name in admin:
         #   cursor.execute("INSERT INTO group_adminOf (username, name) VALUES (?, ?, ?)", 
          #                 (username, name))

        #for key, value in stats.items():
         #   try:
          #      cursor.execute("INSERT INTO stats (key, value) VALUES (?, ?)", (key, value))
          #  except sqlite3.IntegrityError:
          #      print(f"Key '{key}' already exists. Skipping.")  # Handle duplicate keys

        #
        for key, value in profile.items():
            try:
                cursor.execute("INSERT INTO profile (key, value) VALUES (?, ?)", (key, value))
            except sqlite3.IntegrityError:
                print(f"Key '{key}' already exists. Skipping.")  # Handle duplicate keys



        conn.commit()
        conn.close()

    def dump_data_to_sqlite(self, backup_db_path="/mnt/hdd/maittewa/random_deviants_watchers_watching_backup.db"):
        """Dumps the data from the main SQLite database to a backup database."""
        try:
            # Connect to the main database
            conn_main = sqlite3.connect(self.database_path)

            # Connect to the backup database (will be created if it doesn't exist)
            conn_backup = sqlite3.connect(backup_db_path)

            # Backup the 'deviants' table
            conn_main.backup(conn_backup, pages=0, name="main", progress=None)  

            # Close the connections
            conn_backup.close()
            conn_main.close()

            print(f"Data dumped to backup SQLite database: {backup_db_path}")
        except sqlite3.Error as e:
            print(f"SQLite error during data dump: {e}")
        except Exception as e:
            print(f"Error dumping data to SQLite: {e}")


    def run_random_walk(self, num_devs, num_dates):

        """Executes the random walk algorithm."""
        #current_tag = start_tag
        visited_deviants = set()
        deviant_count = 0  # Keep track of processed deviants
       # Establish a single database connection outside the loop
        count_for_reauth = 0
        # Main execution
        start_date = datetime.date(2003, 1, 1)  # 21 years ago
        end_date = datetime.date.today()
        for _ in range(num_dates):
            random_date = start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))

            random_deviants = self.get_random_deviants_from_daily_deviations(num_devs,random_date)
            
            for deviant in random_deviants:
                if deviant not in visited_deviants:
                    visited_deviants.add(deviant)
                    print(f"Fetching data for deviant: {deviant}")  # Print username
                    #watchers, watching = self.get_watchers_and_watching(deviant)
                    #stats, mem, adm, badges, about_devts = self.scrape(deviant)
                    user_profile = self.parse_user_profile(deviant)
                    # Use existing connection and cursor
                    count_for_reauth += 1 
                    #print(deviant, watchers, watching)
                     # Dump data after every 5 deviants
                    #print("Dumping data to database...")
                    #self.dump_data_to_sqlite()
                    print (user_profile)
                    self.store_data(deviant, user_profile)
                    time.sleep(random.uniform(5,15))                    
                    if count_for_reauth % 10 == 0:
                     #   self.refresh_token()
                        count_for_reauth = 0
                        time.sleep(random.uniform(5,15))
        print("Random walk completed.")
       

In [13]:
# Example usage:
client_id = "42096"
client_secret = "97080792c6d30a4178965e41f1ca15de"
TOKEN_URL = "https://www.deviantart.com/oauth2/token"
REDIRECT_URI = "https://www.deviantart.com/oauth2/authorize"


In [14]:
random_walk = DeviantArtRandomWalk(client_id, client_secret)
num_dts = 4000 
num_deviants = 10
random_walk.run_random_walk(num_deviants,num_dts)

Fetching data for deviant: DanLuVisiArt
{'username': 'DanLuVisiArt', 'real_name': '', 'profil_url': 'https://www.deviantart.com/danluvisiart', 'Artist or Not': True, 'tags': '', 'country': 'United States', 'level': 'Professional', 'specialty': 'Digital Art', 'stats': {'user_deviations': 221, 'user_favourites': 2343, 'user_comments': 25092, 'profile_pageviews': 8121575, 'profile_comments': 21259}}
Fetching data for deviant: Thomas-tzar
{'username': 'Thomas-tzar', 'real_name': '', 'profil_url': 'https://www.deviantart.com/thomas-tzar', 'Artist or Not': True, 'tags': '', 'country': 'Unknown', 'level': 'Professional', 'specialty': 'Digital Art', 'stats': {'user_deviations': 30, 'user_favourites': 167, 'user_comments': 42, 'profile_pageviews': 18258, 'profile_comments': 26}}
Fetching data for deviant: creightonwrites
{'username': 'creightonwrites', 'real_name': '', 'profil_url': 'https://www.deviantart.com/creightonwrites', 'Artist or Not': True, 'tags': 'Once!', 'country': 'United States',

KeyboardInterrupt: 

In [42]:
# Start the token refresh loop in a separate thread
refresh_thread = threading.Thread(target=token_refresh_loop)
refresh_thread.daemon = True  # Allow the program to exit even if the thread is running
refresh_thread.start()

NameError: name 'token_refresh_loop' is not defined

In [ ]:
 for related_deviant in watchers:
            cursor.execute("INSERT INTO relationships VALUES (?, ?, ?)",
                           (deviant, related_deviant.username, "watcher"))

        for related_deviant in watching:
            cursor.execute("INSERT INTO relationships VALUES (?, ?, ?)",
                           (deviant, related_deviant.username, "watching"))

In [4]:
def scrape(username):   
        try:
            # --- About Page ---
            about_page_url = f"https://www.deviantart.com/{username}/about"
            about_page_response = requests.get(about_page_url)
            about_page_response.raise_for_status()  # Raise an exception for bad responses
      
            about_page_soup = BeautifulSoup(about_page_response.content, "html.parser")
            id_pattern = re.compile(r"module-\d{10}")
            # Extract About page (adjust the selector based on the page structure)
            aboutpage = about_page_soup.select_one("div.wuU4s._3tnu8._2yPuh")
            userstats_element = about_page_soup.find(id="userstats")  # Userstats
            memberOf = about_page_soup.find('section', id="group_list_members")  
            adminOf = about_page_soup.find('section', id="group_list_admins")
            badges_element = about_page_soup.find(id="badges_activity")
            about_deviant = about_page_soup.find(id=id_pattern)

            userstats = userstats_element.text.strip() if userstats_element else ""
            badges = badges_element.text.strip() if badges_element else ""
            about_devts = about_deviant.text.strip() if about_deviant else ""
            groupmem = memberOf.text.strip() if memberOf else ""
            groupadmin = adminOf.text.strip() if adminOf else ""
            # Find the section with the specific ID
    
            

                
            statistics_data = {}
    
            # Define patterns for each statistic
            patterns = {
            "Pageviews": r"Pageviews(\d+)",
            "Deviations": r"Deviations(\d+)",
            "Watchers": r"Watchers(\d+)",
            "Watching": r"Watching(\d+)",
            "Favourites": r"Favourites(\d+)",
            "Comments Made": r"Comments\sMade(\d+)",  # Handle space in "Comments Made"
            "Comments Received": r"Comments\sReceived(\d+)",  # Handle space in "Comments Received"
            }

            # Extract values using patterns
            for key, pattern in patterns.items():
              match = re.search(pattern, userstats)
              if match:
                statistics_data[key] = int(match.group(1))  # Convert value to integer

            print(about_devts)    
            return statistics_data, groupmem, groupadmin, badges, about_devts
    
        except requests.exceptions.RequestException as e:
            print(f"Error scraping DeviantArt profile: {e}")
            if "403 Client Error" in str(e):  # Check for Forbidden error
                print(f"Waiting for 60 seconds before retrying...")
                time.sleep(60)  # Wait for 60 seconds
                return self.scrape(username)  # Retry the request

In [5]:
scrape('nestry')

nestrySep 8ChileDeviant for 13 yearsHe / HimBadges


({'Pageviews': 2,
  'Deviations': 0,
  'Watchers': 0,
  'Watching': 0,
  'Favourites': 0,
  'Comments Made': 0,
  'Comments Received': 0},
 '',
 '',
 'BadgesGive nestry Their First Badge!Everyone loves a Llama...',
 'nestrySep 8ChileDeviant for 13 yearsHe / HimBadges')

In [6]:
scrape('linked-memories')

Linked-MemoriesArtist  //  Digital ArtDec 9, 1992United KingdomDeviant for 16 yearsVgenFollow me onBadges+34My BioHello!I'm a 31 year old digital illustrator who paints a lot of landscapes❤️ych/adopt account:@linkedx92  --feel free to:use my art as personal wallpaper / screensaveruse it for other non-profitable things (including music videos)you don't need to ask unless you need a commercial licenseFavourite Visual ArtistMy art buddiesFavourite TV ShowsIASIP, HxHFavourite Bands / Musical ArtistsGreen Day, Sub Focus, Nirvana, Hybrid Minds, Primal ScreamFavourite WritersRob McElhenney, Charlie DayFavourite GamesHonkai Star Rail, TLOZFavourite Gaming PlatformPC, SwitchTools of the TradeHuion Kamvas 20, SAIOther Interestspochita


({'Pageviews': 560,
  'Deviations': 977,
  'Watchers': 21,
  'Watching': 735,
  'Favourites': 7,
  'Comments Made': 32,
  'Comments Received': 5},
 'Group MemberCRAdoptablesYCH-BazaarAll-Without-LimitsWallpapersCommunityDigital-DreamscapeAdvertisingActivityGenshinImpact-FanArtAuroraCollectiveVtuberFanClubVtuberArtWonderful-ArtDigital-Art-Fantasyart-dailyU-BUY-WE-SELLGenshin-ImpactLoad more',
 'Group AdminConcept-Addicts',
 "Badges8K Badges20Cake1Winter WonderLAD18Bleeding Love10Whipped Cream11Gold Bar1Platinum Fragment1Shimmering Snowball1Picture Perfect1DeviantArt Tutorial Cat1DeviantArt Originals7KLlama1thoughtART1birthdAy '141I've seen it1Birthday '151truth® Artist Collective1Twist Fate1Khush130 Days of Pride1CreatureWeek20Thursday154Quartz21Emerald19Diamond1ArtFortunes1DeviantArt Tutorials Landscape81Heart1Cupid38Gold Coin1How to Draw a Wolf62Hype1Arc is Long1How to Draw Pikachu233Snowball34Deviously Devoted45Gold Coin 20231Totally Normal1DeviantArt Tutorials12Nice Shot1Drawtober 2

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver

In [10]:
driver = webdriver.Chrome(ChromeDriverManager().install())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 34.4 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of selenium to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install msedge-selenium-tools==3.141.1, msedge-selenium-tools==3.141.2, msedge-selenium-tools==3.141.3, msedge-selenium-tools==3.141.4 and selenium==4.11.2 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested selenium==4.11.2
    msedge-selenium-tools 3.141.4 depends on selenium==3.141
    The user requested selenium==4.11.2
    msedge-selenium-tools 3.141.3 depends on selenium==3.141
    The user requested selenium==4.11.2
    msedge-selenium-tools 3.141.2 depends on selenium==3.141
    The user requested selenium==4.11

In [8]:

usernames = ["username1", "username2", "username3"]

for username in usernames:
    user_url = f"https://www.deviantart.com/{username}/about"
    driver.get(user_url)

    section = driver.find_element(By.CSS_SELECTOR, "group_list_members")
    links = section.find_elements(By.TAG_NAME, "a")
    for link in links:
        link_text = link.text.strip()
        last_word = link_text.split()[-1] if link_text else ""
        print(last_word)  # Or store it in a list or other data structure

    while True:
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "_44rzf._3qeLR.PUjtz"))
            )
            load_more_button.click()

            # Wait for new content to load (adjust the wait time as needed)
            WebDriverWait(driver, 10).until(EC.staleness_of(section))

            # Re-locate the section and extract from new links
            section = driver.find_element(By.CSS_SELECTOR, "group_list_members")
            links = section.find_elements(By.TAG_NAME, "a")
            for link in links:
                # Same extraction logic as before
                link_text = link.text.strip()
                last_word = link_text.split()[-1] if link_text else ""
                print(last_word)

        except requests.exceptions.RequestException as e:
            print(f"Error scraping DeviantArt profile: {e}")
            if "403 Client Error" in str(e):  # Check for Forbidden error
                print(f"Waiting for 60 seconds before retrying...")
                time.sleep(60)  # Wait for 60 seconds

AttributeError: 'str' object has no attribute 'capabilities'

In [54]:
def get_token():
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)
    token = oauth.fetch_token(token_url=TOKEN_URL, client_id=client_id, client_secret=client_secret)
     # Extract the access token
    access_token = token['access_token']
    return access_token

In [55]:
get_token()

'd28931a9eee4fa3752537b9301c968e5072fbf9899aeae7395'

In [11]:
def get_response_rate(response):    
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    elif response.status_code == 404:
        return 'user_done'
    elif response.status_code == 429:
        return 'too_many_requests'
    elif response.status_code == 500:
        return 'server error'
    elif response.status_code == 401:
        return 'get new token'

In [2]:
def get_a_week(numdays):
    date_list=[(datetime.date.today() - datetime.timedelta(days=x)).strftime('%Y-%m-%d') for x in range(0, numdays)]
    return date_list

In [3]:
def get_dailydeviations_info(token, day):
    api_url = f"https://www.deviantart.com/api/v1/oauth2/browse/dailydeviations?access_token={token}"
    response = requests.get(api_url, params={'date': day})
    return response

In [7]:
def get_seed(numdays):
    token=get_token()
    date=random.choice(get_a_week(numdays))
    response=get_dailydeviations_info(token, date)
    j=get_response_rate(response)
    if j=='too_many_requests':
        time.sleep(60)
        j=get_response_rate(response)
    elif j=='get new token': 
        token=get_token()
        j=get_response_rate(response)
    return random.choice([i['author'] for i in j['results']])

In [13]:
user=get_seed(6)['username']

In [14]:
user

'Linked-Memories'

In [112]:
def get_user_profile(username):
    token=get_token()
    #url=f"https://www.deviantart.com/api/v1/oauth2/user/profile/{username}?access_token={token}"
    url=f"https://www.deviantart.com/api/v1/oauth2/user/profile/{username}?access_token={token}"
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error getting profile info: {e}")
        return None
    return response

In [ ]:
def get_user_profile(username):
    token=get_token()
    
    except requests.exceptions.RequestException as e:
        print(f"Error getting profile info: {e}")
        return None
    return response

In [113]:
get_user_profile(user)

<Response [200]>

In [116]:
def parse_user_profile(user):
    s=get_user_profile(user)
    d=json.loads(s.text)
    if 'error' not in d.keys():
        a={'username':user,
        'real_name': d.get('real_name'),
        'profil_url': d['profile_url'],
        'Artist or Not': d['user_is_artist'],   
        'tags': d['tagline'],
        'country': d['country'],
        'level': d['artist_level'],
        'specialty' :d['artist_specialty'],
        'stats': d['stats']}
        return a
    else:
        return 'error'

In [117]:
parse_user_profile(user)

{'username': 'Linked-Memories',
 'real_name': '',
 'profil_url': 'https://www.deviantart.com/linked-memories',
 'Artist or Not': True,
 'tags': 'our passion is our strength',
 'country': 'United Kingdom',
 'level': 'None',
 'specialty': 'Digital Art',
 'stats': {'user_deviations': 977,
  'user_favourites': 7692,
  'user_comments': 32834,
  'profile_pageviews': 560170,
  'profile_comments': 5632}}

In [95]:
def scrape_deviantart_artist(username):
    """Scrapes an artist's About page and extracts hrefs and nested div info."""

    url = f"https://www.deviantart.com/{username}/about" 
    response = requests.get(url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

    soup = BeautifulSoup(response.content, "html.parser")

    # Find the section with the specific ID
    memberOf = soup.find("section", id="group_list_members")  
    adminOf = soup.find("section", id="group_list_admins")
    member_hrefs_and_names = []
    admin_hrefs_and_names = []

    if memberOf:
        # Find the div with the desired class containing the hrefs
        links_div = memberOf.find("div", class_="_2u8NU")  # Replace with actual class name
        if links_div:
            for link in links_div.find_all("a", href=True):
                href = link["href"]
                name_div = link.find("div", class_="_1kDW5")  # Replace with actual nested div class
                name = name_div.text.strip() if name_div else ""
                member_hrefs_and_names.append((href, name))
        else:
            return []
    else:
        return ''
           
    if adminOf:
        # Find the div with the desired class containing the hrefs
        links_div = adminOf.find("div", class_="_2u8NU")  # Replace with actual class name
        if links_div:
            for link in links_div.find_all("a", href=True):
                href = link["href"]
                name_div = link.find("div", class_="_1kDW5")  # Replace with actual nested div class
                name = name_div.text.strip() if name_div else ""
                admin_hrefs_and_names.append((href, name))   

        else:
            return [] 

    else:
        return ''

    return (member_hrefs_and_names,admin_hrefs_and_names)


    # Return an empty list if the div is not found


In [96]:
scrape_deviantart_artist('wolfysilver')

([('https://www.deviantart.com/community', 'Community'),
  ('https://www.deviantart.com/auroracollective', 'AuroraCollective'),
  ('https://www.deviantart.com/brightandcheerful', 'BrightAndCheerful'),
  ('https://www.deviantart.com/traditional-art-draw', 'Traditional-Art-Draw'),
  ('https://www.deviantart.com/exclusively-all-art', 'Exclusively-All-ART'),
  ('https://www.deviantart.com/ratlovers', 'RatLovers'),
  ('https://www.deviantart.com/acrylic-animals', 'Acrylic-Animals'),
  ('https://www.deviantart.com/pot-of-gold', 'Pot-Of-Gold'),
  ('https://www.deviantart.com/help-the-animals', 'help-the-animals'),
  ('https://www.deviantart.com/theblendingstump', 'TheBlendingStump'),
  ('https://www.deviantart.com/ratcommunity', 'RatCommunity'),
  ('https://www.deviantart.com/amazinganimalart', 'AmazingAnimalArt'),
  ('https://www.deviantart.com/pets-of-deviantart', 'Pets-of-deviantART'),
  ('https://www.deviantart.com/da-birds', 'DA-Birds'),
  ('https://www.deviantart.com/creaturesculptartis

In [ ]:
if memberOf:
                # Find the div with the desired class containing the hrefs
                links_div = memberOf.find("div._2u8NU")  # Replace with actual class name
                if links_div:
                    for link in links_div.find_all("a", href=True):
                        href = link["href"]
                        name_div = link.find("div._1kDW5")  # Replace with nested div class
                        name = name_div.text.strip() if name_div else ""
                        member_hrefs_and_names.append((href, name))
                else:
                    print(f"Error fetching group membership for {username}")
                    return []
            else: print('No group memberships')
           
            if adminOf:
                # Find the div with the desired class containing the hrefs
                links_div = adminOf.find("div._2u8NU")  # Replace with actual class name
                if links_div:
                    for link in links_div.find_all("a", href=True):
                        href = link["href"]
                        name_div = link.find("div._1kDW5")  # Replace with actual nested div class
                        name = name_div.text.strip() if name_div else ""
                        admin_hrefs_and_names.append((href, name))   

                else:
                     print(f"Error fetching group admin for {username}")
                     return []
            else:
                 print("Not a group admin")

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv("/mnt/hdd/maittewa/deviants60K_profileAPI.csv")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   username       119 non-null    object 
 1   real_name      0 non-null      float64
 2   profil_url     119 non-null    object 
 3   Artist or Not  119 non-null    bool   
 4   tags           72 non-null     object 
 5   country        119 non-null    object 
 6   level          43 non-null     object 
 7   specialty      48 non-null     object 
 8   stats          119 non-null    object 
dtypes: bool(1), float64(1), object(7)
memory usage: 7.7+ KB
